In [1]:
# Load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('ex4data1.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [2]:
X = data['X']
Y = data['y']

# Training data of 5000 20x20 pixel images
X.shape

(5000, 400)

In [3]:
# We should one-hot encode our label
# We can use a utility function from scikit
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(Y)

y_onehot[0], Y[0]

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([10], dtype=uint8))

## Our Neural Network
* 400 input units (+ the bias unit)
* hidden layer with 25 nodes (+1 bias unit)
* output layer with 10 units

In [4]:
# Functions required to compute the cost
def sigmoid(z):
    return 1 / ((1 + np.exp(-z)))

def forward_propagate(X, theta1, theta2):
    m = X.shape[0]  # number of training samples
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)  # insert our bias unit column
    
    
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)  # compute activation in input layer and add bias column
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [5]:
# implement back propagation
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]  # number of training examples
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape parameter array into parameters for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost for one iteration
    # note that this is unregularized
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
        
    J = J / m
    
    # Regularizing our parameters
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))  
    
    
    # perform back propagation
    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        ht = h[t,:]
        yt = y[t,:]
        
        d3t = ht - yt
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))
        
        delta2 += d3t.T * a2t
        delta1 += (d2t[:,1:]).T * a1t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add gradient regularization
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad
        

In [6]:
# test our implementation of computing cost 
input_size = 400  # 20x20 pixel values
hidden_size = 25
num_labels = 10
learning_rate = 1

# randomly initialize parameters for the whole network
params = (np.random.random(size = ((input_size + 1) * hidden_size) + ((hidden_size + 1) * num_labels)) - 0.5) * 0.25

print(backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate))


(7.0294124987342883, array([  4.99520915e-03,  -8.43684531e-06,  -6.63374624e-07, ...,
         1.54784827e-01,   1.75914972e-01,   1.78003604e-01]))


In [19]:
# train!
from scipy.optimize import minimize

fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),  
                method='TNC', jac=True, options={'maxiter': 250})
fmin 

     fun: 0.3370315444903268
     jac: array([ -9.42437274e-05,   3.78626384e-07,   2.20719239e-08, ...,
         8.76039123e-05,   2.10693335e-05,   5.58748882e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 21
  status: 3
 success: False
       x: array([  4.12896862e-01,   1.89313192e-03,   1.10359619e-04, ...,
        -1.67587177e+00,  -1.89729493e+00,  -2.72924716e+00])

In [24]:
X = np.matrix(X)

theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)  
y_pred = np.array(np.argmax(h, axis=1) + 1)  


In [25]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, Y)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print('accuracy = {}'.format(accuracy * 100))

accuracy = 99.26
